In [ ]:
ENV_NAME = "CliffWalking-v0"
N_ITERATIONS = 3
N_BATCHES = 3
TRAIN_STEPS = 100000
ENV_VAR = []

In [ ]:
"""
Values in an observation

cos(theta1) — cosine of the first joint angle

sin(theta1) — sine of the first joint angle

cos(theta2) — cosine of the second joint angle

sin(theta2) — sine of the second joint angle

theta1_dot — angular velocity of the first joint

theta2_dot — angular velocity of the second joint

"""

In [ ]:
#### Implemented on python. No wrapper needed

#!apt-get install -y swig

#!pip install imageio box2d-py gymnasium[box2d]

!pip install stable-baselines3[extra]

In [ ]:
# import libraries
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import imageio
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import EvalCallback, BaseCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from IPython.display import Image, display,clear_output
from PIL import Image as PILImage, ImageDraw, ImageFont
from gymnasium.wrappers import TimeLimit
from collections import Counter

import os, builtins, math

In [ ]:
# Lookup which scenarios in gymnasium are well suited for DQN (linear and lowdimensional (2D))

for env_spec in gym.registry.values():
    if hasattr(env_spec, "entry_point"):
        try:
            env = gym.make(env_spec.id)
            if isinstance(env.action_space, gym.spaces.Discrete):
                print(env_spec.id)
        except:
            continue

In [ ]:
# The structure looks something like this:
"""
DummyVecEnv
└── Monitor
    └── TimeLimit
        └── CartPoleEnv
"""

#Custom reward system wrapper

class RewardShapingWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        done = terminated or truncated


        # Example: currently no modification, just pass through
        # You can customize reward shaping here:
        # if reward == -100:
        #     reward = -10  # soften cliff penalty, for instance

        return obs, reward, terminated, truncated, info

#Custom Live Render Function

class LiveRenderCallback(BaseCallback):
    def __init__(self, render_freq=1000, verbose=0):
        super().__init__(verbose)
        self.render_freq = render_freq

    def _on_step(self) -> bool:
        if self.n_calls % self.render_freq == 0:
            shot = self.training_env.render(mode="rgb_array")
            clear_output(wait=True)
            plt.imshow(shot)
            plt.axis("off")
            plt.show()
        return True


#We need to umwrap the structure until we reach the instance of monitor
def unwrap_monitor(env):
    """Recursively unwrap env until we find a Monitor."""
    current = env
    while not isinstance(current, Monitor):
        if hasattr(current, 'env'):
            current = current.env
        else:
            raise ValueError("Monitor wrapper not found in the environment stack.")
    return current

# ✅ A Gym environment → wrapped in a Monitor(logs metrics, rewards... for episodes) →
# then wrapped in a Reward Shaper  →  then wrapped in a VecEnv (like DummyVecEnv)

def make_env():
    def _init():
        env = gym.make(ENV_NAME, render_mode="rgb_array")
        env = RewardShapingWrapper(env)
        env = Monitor(env)  # Wrap with Monitor first
        return env
    return DummyVecEnv([_init])

def test_agent(env=None, model=None, iterations=1, env_name="env", annotate_fn=None, font=None):
    import builtins
    if env is None:
        env = globals().get("env")
    if model is None:
        model = globals().get("model")

    if env is None or model is None:
        raise ValueError("env and model must be defined or passed explicitly")

    if font is None:
        try:
            font = ImageFont.truetype("DejaVuSans.ttf", 14)
        except:
            font = ImageFont.load_default()

    for i in range(iterations):
        obs = env.reset()
        step = 0
        total_reward = 0
        action_counter = Counter()
        frames = []
        done = False

        while not done:
            frame = env.render()

            img = PILImage.fromarray(frame)
            draw = ImageDraw.Draw(img)

            # Predicts next action based on the current observation
            action, _ = model.predict(obs, deterministic=True)
            # Takes the step (action) and returns the new observation, rewads and state
            obs, rewards, dones, infos = env.step(action)
            done = dones[0]  # Since it's a single environment [0] inside DummyVecEnv
            reward = rewards[0]
            info = infos[0]

            total_reward += reward
            action_counter[action.item()] += 1
            step += 1

            # Unwrap observation if needed
            if isinstance(obs, np.ndarray) and obs.ndim == 2:
                obs_values = obs[0]
            else:
                obs_values = obs

            # ✍️ Annotation via custom function or fallback
            if annotate_fn:
                text_lines = annotate_fn(obs_values, step, total_reward, info)
            else:
                text_lines = [
                    f"Step: {step}",
                    f"Total Reward: {total_reward:.2f}"
                ]


            y_offset = 10
            for line in text_lines:
                draw.text((10, y_offset), line, fill=(0, 0, 0), font=font)
                y_offset += 15

            frames.append(np.array(img))

        # Save and display GIF
        gif_path = f"videos/{env_name.lower()}_episode_{i}_agent.gif"
        imageio.mimsave(gif_path, frames, fps=30)
        display(Image(filename=gif_path))

        # Unwrap and plot rewards
        monitor_env = unwrap_monitor(env.envs[0])
        results = monitor_env.get_episode_rewards()

        env.close()

        print(f"\n▶️ Episode {i}")
        print(f"Total reward: {total_reward:.2f}")
        print(f"Episode length: {step} steps")
        print("Action distribution:", dict(action_counter))

        plt.figure(figsize=(12, 6))
        plt.plot(results)
        plt.xlabel("Episode")
        plt.ylabel("Reward")
        plt.title("Training Rewards per Episode")
        plt.show()

        plt.bar(action_counter.keys(), action_counter.values())
        plt.xlabel("Action")
        plt.ylabel("Frequency")
        plt.title("Action Distribution")
        plt.show()

def train_multiple_batches(model_class, env_func, total_timesteps=300_000, n_batches=3):
    all_results = []

    for batch_idx in range(n_batches):
        print(f"\n--- Starting batch {batch_idx + 1} ---")
        env = env_func()
        model = model_class("MlpPolicy", env, verbose=1)
        model.learn(total_timesteps=total_timesteps)

        monitor_env = unwrap_monitor(env.envs[0])
        batch_rewards = monitor_env.get_episode_rewards()
        all_results.append(batch_rewards)

        env.close()

    return all_results

def plot_batch_results(all_results):
    max_len = max(len(r) for r in all_results)
    padded = [np.pad(r, (0, max_len - len(r)), constant_values=np.nan) for r in all_results]
    stacked = np.vstack(padded)

    mean_rewards = np.nanmean(stacked, axis=0)
    std_rewards = np.nanstd(stacked, axis=0)

    plt.figure(figsize=(12,6))
    plt.plot(mean_rewards, label='Mean reward')
    plt.fill_between(range(max_len), mean_rewards - std_rewards, mean_rewards + std_rewards, alpha=0.2)
    plt.xlabel("Episode")
    plt.ylabel("Reward")
    plt.title("Training Rewards Across Batches (Mean ± Std)")
    plt.legend()
    plt.show()

def frozenlake_annotate_fn(obs, step, total_reward, info):
    text_lines = [
        f"Step: {step}",
        f"Total Reward: {total_reward:.2f}",
        f"Position: {obs}",  # obs is a scalar tile index
    ]
    return text_lines


In [ ]:
# Create and wrap environment (Monitor helps log rewards)

env = make_env()

# Create eval environment (for callback evaluation)
eval_env = make_env()

# Create directory to save models and videos
os.makedirs("models", exist_ok=True)
os.makedirs("videos", exist_ok=True)

# Create the DQN agent with a Multi Layer Perceptron
model = DQN(
    policy="MlpPolicy",
    env=env,
    learning_rate=1e-3,  # Slightly higher to help early learning
    buffer_size=100_000,  # More transitions to remember rare successes
    learning_starts=1_000,
    batch_size=64,
    tau=1.0,
    gamma=0.99,
    train_freq=4,
    target_update_interval=500,
    exploration_initial_eps=1.0,
    exploration_final_eps=0.1,
    exploration_fraction=0.2,  # Explore for first 20% of training
    verbose=1,
    tensorboard_log="./frozenlake_dqn_tensorboard/",
    policy_kwargs=dict(net_arch=[128, 128]),  # Slightly deeper net
)

# Setup evaluation callback to save best model during training
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path="./models/best_model",
    log_path="./models/",
    eval_freq=5000,
    deterministic=True,
    render=False,
)

# Train the model (you can increase total_timesteps for better results)
model.learn(total_timesteps=TRAIN_STEPS, callback=[eval_callback, LiveRenderCallback()])

# Save the final model
model.save(f"models/dqn_{ENV_NAME}_final")

# Load best saved model
model = DQN.load("./models/best_model/best_model.zip", env=env)

test_agent()

In [ ]:
test_agent(env=env, model=model, iterations=N_ITERATIONS, env_name="FrozenLake", annotate_fn=frozenlake_annotate_fn)

In [ ]:
batch_results = train_multiple_batches(DQN, make_env, total_timesteps=TRAIN_STEPS, n_batches=N_BATCHES)
plot_batch_results(batch_results)

In [ ]:
"""
+-------------+ (step)   +-----------------+          +--------------------+
|             | action   |                 | new obs  |                    |
|   Agent     +--------->+   Environment   +--------->+     Agent again    |
|  (predict)  |          |                 | reward   |                    |
+-------------+          +-----------------+  done?   +--------------------+
                                  ^                               |
                                  |<-------------[loop]-----------+

"""

## Hyperparameter tuning with manual search

import itertools

def train_dqn_with_params(env, params):
    model = DQN(
        "MlpPolicy",
        env,
        learning_rate=params["learning_rate"],
        batch_size=params["batch_size"],
        gamma=params["gamma"],
        verbose=0,
    )
    model.learn(total_timesteps=10000)
    return model

search_space = {
    "learning_rate": [1e-3, 5e-4],
    "batch_size": [32, 64],
    "gamma": [0.95, 0.99]
}

def evaluate_model(model, env, n_episodes=5):
    rewards = []
    for _ in range(n_episodes):
        obs = env.reset()
        done = False
        total_reward = 0
        while not done:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            total_reward += reward
        rewards.append(total_reward)
    return sum(rewards) / n_episodes

best_params = None
best_reward = -float('inf')
results = []

for values in itertools.product(*search_space.values()):
    params = dict(zip(search_space.keys(), values))
    print(f"Training with params: {params}")
    model = train_dqn_with_params(env, params)
    avg_reward =  (model, env)
    print(f"Average reward: {avg_reward}")

    results.append((params, avg_reward))
    if avg_reward > best_reward:
        best_reward = avg_reward
        best_params = params

print(f"Best params: {best_params} with average reward {best_reward}")

In [ ]:
## Hyperparameter tuning with random search

import random

def random_search(env, search_space, n_trials=10):
    results = []
    for _ in range(n_trials):
        params = {k: random.choice(v) for k, v in search_space.items()}
        print(f"Training with params: {params}")
        model = train_dqn_with_params(env, params)
        # Evaluate model here
        results.append((params, model))
    return results

# Use same search_space as above
results = random_search(env, search_space)


In [ ]:
!pip install optuna

In [ ]:
## Hyperparameter tuning automated with optuna

import optuna
from stable_baselines3.common.evaluation import evaluate_policy

def objective(trial):
    params = {
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e-3),
        "batch_size": trial.suggest_categorical("batch_size", [32, 64, 128]),
        "gamma": trial.suggest_uniform("gamma", 0.9, 0.999),
    }
    model = DQN("MlpPolicy", env, **params, verbose=0)
    model.learn(total_timesteps=10000)
    mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
    return mean_reward

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best params:", study.best_params)
